# MCF Instances

## Formulations of MCNF problems

Source: [Input Format for Multicommodity Flow Problems](ftp://ftp.cs.wisc.edu/math-prog/cpo-dataset/networks/multicommodity/Jones_Lustig_Farv/format.tex)

The JL format for MCNF problems defines these different ways to formulate a problem:
- **origin-destination problem (ODP)** - The commodity is defined as a product $k$, that travels between a specific *origin* $s$ and a specifig *destination* $t$. The commodity is then a triple $$\kappa = (k, s, t)$$
- **destination specific problem (DSP)** - The commodity is defined as a product that travels to a specific destination from multiple origins, or vice versa, from a specific origin to multiple destinations. The commodity is then a tuple $$\kappa = (k, t) \quad \text{or} \quad \kappa = (k, s)$$
- **product specific problem (PSP)** - The commodity is defined as a product that must travel through the network from multiple origins to multiple destinations. The commodity is then the singleton $$\kappa = k$$.

Problem instances for the Linear Multicomodity Flow Problems are taken from
[Multicommodity Problem Instances](http://groups.di.unipi.it/optimize/Data/MMCF.html#MNetGen)

In [2]:
# path to data directory
instances_path = "../../data/instances"

## Mnetgen

**Mnetgen** is a random generator for Multicommodity Min Cost Flow instances. The format of the problem instances is as follows.

A problem instance consist of 4 files:
- **Mutual capacity file** (``*.mut``)
- **Arc file** (``*.arc``)
- **Node supply file** (``*.nod`` or ``*.sup`` )
- **Problem description file** (``*.nod``)